https://m.blog.naver.com/PostView.nhn?blogId=khyunji90&logNo=220166480550&proxyReferer=https:%2F%2Fwww.google.com%2F

In [78]:
import pandas as pd
df=pd.DataFrame([
    ['green','M','10.1','class1'],
    ['red','L','13.5','class2'],
    ['blue','XL','15.3','class1']
])
df.columns=['color','size','price','classlabel']

In [79]:
df

,color,size,price,classlabel
0,green,M,10.1,class1
1,red,L,13.5,class2
2,blue,XL,15.3,class1


In [80]:
size_mapping={
    'XL':3,
    'L':2,
    'M':1
}

In [81]:
df['size']=df['size'].map(size_mapping)

In [82]:
df

,color,size,price,classlabel
0,green,1,10.1,class1
1,red,2,13.5,class2
2,blue,3,15.3,class1


In [13]:
inv_size_mapping={v:k for k,v in size_mapping.items()}
df['size']=df['size'].map(inv_size_mapping)

In [14]:
df

,color,size,price,classlabel
0,green,M,10.1,class1
1,red,L,13.5,class2
2,blue,XL,15.3,class1


In [15]:
import numpy as np
class_mapping={label:idx for idx,label in enumerate(np.unique(df['classlabel']))}


In [16]:
class_mapping

{'class1': 0, 'class2': 1}

In [17]:
df['classlabel']=df['classlabel'].map(class_mapping)

In [18]:
df

,color,size,price,classlabel
0,green,M,10.1,0
1,red,L,13.5,1
2,blue,XL,15.3,0


In [19]:
inv_class_mapping={v:k for k,v in class_mapping.items()}
df['classlabel']=df['classlabel'].map(inv_class_mapping)
df

,color,size,price,classlabel
0,green,M,10.1,class1
1,red,L,13.5,class2
2,blue,XL,15.3,class1


In [20]:
inv_class_mapping

{0: 'class1', 1: 'class2'}

In [21]:
from sklearn.preprocessing import LabelEncoder
class_le=LabelEncoder()
y=class_le.fit_transform(df['classlabel'].values)
y

array([0, 1, 0])

In [24]:
class_le.inverse_transform(y)

array(['class1', 'class2', 'class1'], dtype=object)

In [94]:
x=df[['color','size','price']].values
class_le=LabelEncoder()
x[:,0]=class_le.fit_transform(x[:,0])

In [95]:
x

array([[1, 1, '10.1'],
       [2, 2, '13.5'],
       [0, 3, '15.3']], dtype=object)

In [90]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OrdinalEncoder
ord_enc=OrdinalEncoder(dtype=np.int)
col_trans=ColumnTransformer([('ord_enc',ord_enc,['color'])])
X_trains=col_trans.fit_transform(df)
X_trains

array([[1],
       [2],
       [0]])

In [93]:
df

,color,size,price,classlabel
0,green,1,10.1,class1
1,red,2,13.5,class2
2,blue,3,15.3,class1


In [92]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OrdinalEncoder
ord_enc=OrdinalEncoder(dtype=np.int)
col_trans=ColumnTransformer([('ord_enc',ord_enc,['size'])])
X_trains=col_trans.fit_transform(df)
X_trains

array([[0],
       [1],
       [2]])

In [71]:
col_trans.named_transformers_['ord_enc'].inverse_transform(X_trains)

array([['green'],
       ['red'],
       ['blue']], dtype=object)

OrdinalEncoder는 순서가 있는 트겅에 적용하는 클래스가 아니라 순서가 없는 범주형 특성을 정수로 변환하는 클래스

문제: green는 red보다 크고 blue보다 작다고 가정,하지만 알고리즘이 의미 있는 결과를 만들 수 있다.

위 문제를 해결하기 위해 원-핫-인코딩(one-hot-encoding)기법을 사용

순서 없는 특성에 들어 있는 고유한 값마다 새로운 더미(dummy)특성을 만드는 것

In [85]:
from sklearn.preprocessing import OneHotEncoder

In [98]:
ohe=OneHotEncoder(categories='auto')
col_trans=ColumnTransformer([('ohe',ohe,[0])],remainder='passthrough')
col_trans.fit_transform(x)

array([[0.0, 1.0, 0.0, 1, '10.1'],
       [0.0, 0.0, 1.0, 2, '13.5'],
       [1.0, 0.0, 0.0, 3, '15.3']], dtype=object)

In [109]:
pd.get_dummies(df[['color','price','size']])


,size,color_blue,color_green,color_red,price_10.1,price_13.5,price_15.3
0,1,0,1,0,1,0,0
1,2,0,0,1,0,1,0
2,3,1,0,0,0,0,1


원-핫 인코딩된 데이터셋을 사용할때 다중 공선성(multicollinearity)주의

특성 간의 상관관계가 높으면 역행렬을 계한하기 어려워 수치적으로 불안해진다. 

변수 간의 상관관계를 감소하려면 원-핫 인코딩 된 배열에서 특성 열 하나 삭제

In [104]:
pd.get_dummies(df[[
    'price','color','size'
]],drop_first=True)

,size,price_13.5,price_15.3,color_green,color_red
0,1,0,0,1,0
1,2,1,0,0,1
2,3,0,1,0,0


https://m.blog.naver.com/PostView.nhn?blogId=vnf3751&logNo=220833952857&proxyReferer=https:%2F%2Fwww.google.com%2F

https://mindscale.kr/course/basic-stat-python/13/

In [171]:
ohe=OneHotEncoder(categories='auto')
col_trans=ColumnTransformer([('ohe',ohe,[0])],remainder='passthrough')
col_trans.fit_transform(x)[:,1:]

array([[1.0, 0.0, 1, '10.1'],
       [0.0, 1.0, 2, '13.5'],
       [0.0, 0.0, 3, '15.3']], dtype=object)

In [172]:
df=pd.read_csv('https://archive.ics.uci.edu/ml/'
              'machine-learning-databases/wine/wine.data',header=None)

In [173]:
df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,1,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065
1,1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050
2,1,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185
3,1,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480
4,1,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173,3,13.71,5.65,2.45,20.5,95,1.68,0.61,0.52,1.06,7.70,0.64,1.74,740
174,3,13.40,3.91,2.48,23.0,102,1.80,0.75,0.43,1.41,7.30,0.70,1.56,750
175,3,13.27,4.28,2.26,20.0,120,1.59,0.69,0.43,1.35,10.20,0.59,1.56,835
176,3,13.17,2.59,2.37,20.0,120,1.65,0.68,0.53,1.46,9.30,0.60,1.62,840


In [174]:
df_wine=df

In [175]:
df_wine

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,1,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065
1,1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050
2,1,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185
3,1,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480
4,1,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173,3,13.71,5.65,2.45,20.5,95,1.68,0.61,0.52,1.06,7.70,0.64,1.74,740
174,3,13.40,3.91,2.48,23.0,102,1.80,0.75,0.43,1.41,7.30,0.70,1.56,750
175,3,13.27,4.28,2.26,20.0,120,1.59,0.69,0.43,1.35,10.20,0.59,1.56,835
176,3,13.17,2.59,2.37,20.0,120,1.65,0.68,0.53,1.46,9.30,0.60,1.62,840


In [176]:
df_wine.columns=['Class label','Alcohol','Malic acid','Ash','Alcalinity of ash','Magnesium','Total phenols','Flacanoids',
                'Nonflacanoid phenols','Proanthocyanins','Color intensity','Hue','0D280/0D315 of diluted wines','Proline']

In [177]:
df_wine

,Class label,Alcohol,Malic acid,Ash,Alcalinity of ash,Magnesium,Total phenols,Flacanoids,Nonflacanoid phenols,Proanthocyanins,Color intensity,Hue,0D280/0D315 of diluted wines,Proline
0,1,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065
1,1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050
2,1,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185
3,1,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480
4,1,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173,3,13.71,5.65,2.45,20.5,95,1.68,0.61,0.52,1.06,7.70,0.64,1.74,740
174,3,13.40,3.91,2.48,23.0,102,1.80,0.75,0.43,1.41,7.30,0.70,1.56,750
175,3,13.27,4.28,2.26,20.0,120,1.59,0.69,0.43,1.35,10.20,0.59,1.56,835
176,3,13.17,2.59,2.37,20.0,120,1.65,0.68,0.53,1.46,9.30,0.60,1.62,840


In [178]:
print("클래스 레이블",np.unique(df_wine['Class label']))

클래스 레이블 [1 2 3]


In [179]:
from sklearn.model_selection import train_test_split
X,y=df_wine.iloc[:,1:],df_wine.iloc[:,0]
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=0,stratify=y)

In [180]:
print(X.shape)
print(X_train.shape)
print(X_test.shape)

(178, 13)
(124, 13)
(54, 13)


In [181]:
print(y_train.shape)
print(y_test.shape)

(124,)
(54,)


# 특성 스케일 맞추기
#### 전처리 파이프 라인에서 잊어버리기 쉽지만 중요한 단계.

결정트리와 랜덤포레스트는 특성 스케일 조정에 대해 걱정할 필요없는 몇없는 알고리즘---스케일에 영향 받지않는다.

경사하강법알고리즘을 구현하면서 보았듯이 대부분의 머신 러닝과 최적화 알고리즘은 특성의 스케일이 같은데 훨씬 성능이 좋다.

#  대표적인 방법은 정규화(Normalization), 표준화(standardization)이 있다.

대부분의 정규화는 [0,1] 범위에 맞추는 것을 의미. 최소-최대-스케일 변화(min-max-scaling)의 특별한 경우


In [182]:
from sklearn.preprocessing import MinMaxScaler
mms=MinMaxScaler()
X_train_norm=mms.fit_transform(X_train)
X_test_norm=mms.transform(X_test)

In [183]:
X_train_norm

array([[0.64619883, 0.83201581, 0.4248366 , ..., 0.45744681, 0.28571429,
        0.19400856],
       [0.6871345 , 0.15612648, 0.65359477, ..., 0.81914894, 0.63369963,
        0.68259629],
       [0.67836257, 0.15019763, 0.65359477, ..., 0.75531915, 0.52747253,
        0.71825963],
       ...,
       [0.72222222, 0.84980237, 0.34640523, ..., 0.10638298, 0.02197802,
        0.09771755],
       [0.16081871, 0.06916996, 0.39215686, ..., 0.54255319, 0.68131868,
        0.43366619],
       [0.37719298, 0.61857708, 0.45751634, ..., 0.75531915, 0.68131868,
        0.13195435]])

In [184]:
X_train_norm

array([[0.64619883, 0.83201581, 0.4248366 , ..., 0.45744681, 0.28571429,
        0.19400856],
       [0.6871345 , 0.15612648, 0.65359477, ..., 0.81914894, 0.63369963,
        0.68259629],
       [0.67836257, 0.15019763, 0.65359477, ..., 0.75531915, 0.52747253,
        0.71825963],
       ...,
       [0.72222222, 0.84980237, 0.34640523, ..., 0.10638298, 0.02197802,
        0.09771755],
       [0.16081871, 0.06916996, 0.39215686, ..., 0.54255319, 0.68131868,
        0.43366619],
       [0.37719298, 0.61857708, 0.45751634, ..., 0.75531915, 0.68131868,
        0.13195435]])

In [185]:
from sklearn.preprocessing import StandardScaler
stdsc=StandardScaler()
X_train_std=stdsc.fit_transform(X_train)
X_test_std=stdsc.transform(X_test)

In [186]:
X_train_std

array([[ 0.71225893,  2.22048673, -0.13025864, ..., -0.20017028,
        -0.82164144, -0.62946362],
       [ 0.88229214, -0.70457155,  1.17533605, ...,  1.33982592,
         0.54931269,  1.47568796],
       [ 0.84585645, -0.73022996,  1.17533605, ...,  1.06806189,
         0.1308109 ,  1.62934866],
       ...,
       [ 1.02803489,  2.29746195, -0.5778911 , ..., -1.69487249,
        -1.86068037, -1.04434751],
       [-1.30384913, -1.08089484, -0.31677217, ...,  0.16218176,
         0.73691694,  0.40313628],
       [-0.40510216,  1.29678411,  0.05625489, ...,  1.06806189,
         0.73691694, -0.89683324]])

In [187]:
X_test_std

array([[ 8.94437367e-01, -3.88117877e-01,  1.10073064e+00,
        -8.12017114e-01,  1.13201117e+00,  1.09807851e+00,
         7.12041017e-01,  1.81013423e-01,  6.62804643e-02,
         5.12859235e-01,  7.96297849e-01,  4.48295020e-01,
         1.90593792e+00],
       [-1.04879931e+00, -7.72993966e-01,  5.41190056e-01,
        -2.40938809e-01,  3.49414498e-01, -7.07219221e-01,
        -3.08121293e-01,  6.76138376e-01, -1.03520519e+00,
        -9.06567274e-01,  2.24570604e+00, -5.61881713e-01,
        -1.22874035e+00],
       [ 1.64744158e+00, -4.13776283e-01,  2.42768413e-01,
        -1.56343594e+00, -2.19746720e-01,  2.84881333e-01,
         4.47191186e-01, -5.61674005e-01,  5.74658458e-01,
         6.79643589e-02,  2.98063782e-01,  1.38631627e+00,
         1.64471473e+00],
       [ 2.02159300e-01, -4.90751501e-01, -9.13615453e-01,
        -2.58536553e+00,  6.48338887e-02,  5.28840485e-01,
         6.82613258e-01, -8.91757306e-01, -6.11556861e-01,
        -3.00662824e-01,  4.33945800e

In [189]:
ex=np.array([0,1,2,3,4,5])
print('표준화: ',(ex-ex.mean())/ex.std())
print('정규화: ',(ex-ex.min())/(ex.max()-ex.min()))

표준화:  [-1.46385011 -0.87831007 -0.29277002  0.29277002  0.87831007  1.46385011]
정규화:  [0.  0.2 0.4 0.6 0.8 1. ]
